# Evaluate the MNIST model by loading the weights

This is an example to demonstrate how we can perform predictions by loading a trained model

In [1]:
import warnings
warnings.filterwarnings("ignore", message = "numpy.dtype size changed")

import tensorflow as tf
from zoo import init_nncontext
from zoo.pipeline.api.net import TFDataset, TFPredictor
import numpy as np
import sys
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from bigdl.dataset import mnist
from bigdl.dataset.transformer import *

import mnist_utilities_workshop

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Define the model

In [2]:
data = Input(shape=[28, 28, 1])

x = Flatten()(data)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=data, outputs=predictions)

## Load the pretrained model

In [ ]:
model.load_weights("mnist_keras.h5")

## Build the predictor

In [3]:
sc = init_nncontext()
        # get data, pre-process and create TFDataset
(images_data, labels_data) = mnist_utilities_workshop.ws_read_data_sets("/opt/work/ODSC-east-2019/datasets/mnist/", "test")
image_rdd = sc.parallelize(images_data[:100])
labels_rdd = sc.parallelize(labels_data[:100])
rdd = image_rdd.zip(labels_rdd).map(lambda rec_tuple: [normalizer(rec_tuple[0], mnist.TRAIN_MEAN, mnist.TRAIN_STD)])

dataset = TFDataset.from_rdd(rdd,
                                     names=["features"],
                                     shapes=[[28, 28, 1]],
                                     types=[tf.float32],
                                     batch_per_thread=20
                                     )
predictor = TFPredictor.from_keras(model, dataset)

('Extracting', '/tmp/mnist/t10k-images-idx3-ubyte.gz')
('Extracting', '/tmp/mnist/t10k-labels-idx1-ubyte.gz')
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
creating: createTFNet


## Predict

In [4]:
accuracy = predictor.predict().zip(labels_rdd).map(lambda x: np.argmax(x[0]) == x[1]).mean()
print("predict accuracy is %s" % accuracy)

predict accuracy is 0.99
